
# Large-scale circuit benchmarks

This notebook benchmarks complex circuit generators on multiple backends and QuASAr's planner. Small and large instances highlight when hybrid planning outperforms single-method simulations.



## Environment

Install required packages and load repository root so that `benchmarks` utilities are importable.

```bash
pip install -e .[test]
pip install pandas seaborn jupyter
```


In [ ]:

import sys, platform, quasar
print('Python', sys.version)
print('Platform', platform.platform())
print('QuASAr', quasar.__version__)



## Benchmark setup

Each circuit generator from `large_scale_circuits.py` is instantiated at two scales. The circuits are executed on pure backends – dense statevector, Stim's tableau simulator, the tensor-network MPS backend and the decision-diagram backend – and via QuASAr's planner. Runtimes and memory consumption are recorded for comparison.


In [ ]:

from __future__ import annotations
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

from runner import BenchmarkRunner
from quasar import SimulationEngine
from quasar.cost import Backend
import large_scale_circuits as lsc


In [ ]:

# Circuit configurations: small and large parameter sets
circuit_builders = {
    'ripple_carry': lsc.ripple_carry_modular_circuit,
    'surface_code': lsc.surface_code_cycle,
    'grover': lsc.grover_with_oracle_circuit,
    'qaoa': lsc.deep_qaoa_circuit,
    'phase_est': lsc.phase_estimation_classical_unitary,
}

configurations = {
    'ripple_carry': {
        'small': dict(bit_width=4, modulus=None, arithmetic='cdkm'),
        'large': dict(bit_width=8, modulus=None, arithmetic='cdkm'),
    },
    'surface_code': {
        'small': dict(distance=3, rounds=1),
        'large': dict(distance=5, rounds=2),
    },
    'grover': {
        'small': dict(n_qubits=4, oracle_depth=2, iterations=1),
        'large': dict(n_qubits=8, oracle_depth=4, iterations=1),
    },
    'qaoa': {
        'small': dict(graph=nx.cycle_graph(4), p_layers=2),
        'large': dict(graph=nx.cycle_graph(8), p_layers=4),
    },
    'phase_est': {
        'small': dict(eigen_qubits=2, precision_qubits=2, classical_depth=1),
        'large': dict(eigen_qubits=3, precision_qubits=3, classical_depth=2),
    },
}


In [ ]:

backends = [Backend.STATEVECTOR, Backend.TABLEAU, Backend.MPS, Backend.DECISION_DIAGRAM]
runner = BenchmarkRunner()
engine = SimulationEngine()


In [ ]:

records = []
for name, builder in circuit_builders.items():
    for size, params in configurations[name].items():
        circ = builder(**params)
        for backend in backends:
            rec = runner.run_quasar_multiple(circ, engine, backend=backend, repetitions=3)
            rec.update({'circuit': name, 'size': size, 'qubits': circ.num_qubits})
            records.append(rec)
        rec = runner.run_quasar_multiple(circ, engine, repetitions=3)
        rec.update({'circuit': name, 'size': size, 'qubits': circ.num_qubits})
        records.append(rec)

df = pd.DataFrame(records)
df



### Runtime and memory comparison

Lines connect the small and large instances for each circuit family. Cross-over points mark where QuASAr becomes faster than a single-method backend.


In [ ]:

sns.set_theme(style='whitegrid')

# Plot runtime
runtime_plot = sns.relplot(
    data=df,
    x='qubits', y='run_time_mean', hue='framework', style='framework',
    col='circuit', col_wrap=2, kind='line', facet_kws={'sharey': False}, marker='o'
)
runtime_plot.set(xscale='log', yscale='log')
plt.show()

# Plot peak memory during run phase
memory_plot = sns.relplot(
    data=df,
    x='qubits', y='run_peak_memory_mean', hue='framework', style='framework',
    col='circuit', col_wrap=2, kind='line', facet_kws={'sharey': False}, marker='o'
)
memory_plot.set(xscale='log', yscale='log')
plt.show()



### Cross-over estimation

For each circuit family we estimate the qubit count at which QuASAr outperforms a given backend. The estimate assumes linear scaling between the sampled small and large sizes.


In [ ]:

crossovers = []
for name in circuit_builders:
    sub = df[(df['circuit'] == name)].pivot_table(
        index='framework', columns='size', values=['run_time_mean', 'qubits']
    )
    if ('run_time_mean', 'small') not in sub.columns:
        continue
    q_small = sub['qubits', 'small'].iloc[0]
    q_large = sub['qubits', 'large'].iloc[0]
    for backend in sub.index:
        if backend == 'quasar':
            continue
        if ('run_time_mean', 'large') not in sub.loc[[backend, 'quasar'], :].columns:
            continue
        qs = sub.loc['quasar', ('run_time_mean', 'small')]
        ql = sub.loc['quasar', ('run_time_mean', 'large')]
        bs = sub.loc[backend, ('run_time_mean', 'small')]
        bl = sub.loc[backend, ('run_time_mean', 'large')]
        diff_small = qs - bs
        diff_large = ql - bl
        if diff_small > 0 and diff_large < 0:
            cross_q = q_small + (q_large - q_small) * diff_small / (diff_small - diff_large)
            crossovers.append({'circuit': name, 'backend': backend, 'crossover_qubits': cross_q})

crossovers_df = pd.DataFrame(crossovers)
crossovers_df



## Save results

Raw benchmark data and crossover estimates are stored in `benchmarks/results/` for reproducibility.


In [ ]:

result_dir = Path('../results')
result_dir.mkdir(exist_ok=True)

df.to_json(result_dir / 'large_scale_circuits_results.json', orient='records', indent=2)
if not crossovers_df.empty:
    crossovers_df.to_csv(result_dir / 'large_scale_circuits_crossovers.csv', index=False)

{'results_path': str(result_dir.resolve())}



## Reproducing this notebook

1. Install dependencies as described in the *Environment* section.
2. Start Jupyter and open `benchmarks/notebooks/large_scale_circuits.ipynb`.
3. Run all cells to generate the results and plots.
